This notebook will create the dataset(s) necessary to hunt our most Bayesian game.
This will require:
1) A dataset, along with counts (for probability calculation) of the order of events for every bill prior to 2018, labeled t1 through tn
2) A dataset that represents the additional feature data for each bill at each stage (docVec cluster, sponsor, etc).
    a) If possible, comb news sources from around that time for a sentiment analysis
    b) Maybe, for fun, get data from the major NH newspapers with editorials and the like?
3) Merge the two data sets

This data set will be used to calculate the Bayesian probability at the DocCluster level.

See Data_Dictionary.pdf for the data structure of for our data.
    

In [2]:
# This cell will accumulate a list of urls from legiscan that will be used for the text extraction
from google.cloud import storage
import json
import pickle
import string
import pandas as pd
import dateutil
from datetime import date
import re
from datetime import datetime

In [2]:
#import dnspython
from pymongo import MongoClient


user = "CEDC_Quigley" #environ.get("OVERLOOK_INTEGRATION_USER")
password = "lpP9pCcDxQ2Y0tv8" #environ.get("OVERLOOK_INTEGRATION_PWORD")

client = MongoClient('mongodb+srv://ridgebaseaq-docdb-dqrxk.mongodb.net/test'
                        ,username=user
                        ,password=password
                        ,authSource='admin'
                        ,authMechanism='SCRAM-SHA-1')

database = client['legislative_documents'] #'ma_rec_test'
collection = database['legiscan_bill_metadata'] #bill_meta

bill_data = []

for bl in collection.find({'state':"US"}).batch_size(30):
    bill_data.append(bl)

In [3]:
with open('raw_federal_metadata09_20.list','wb') as f:
    pickle.dump(bill_data, f)

In [3]:
with open('raw_federal_metadata09_20.list','rb') as f:
    bill_data = pickle.load(f)

In [4]:
#do some analysis on the bill data
year_count = {}
count = 0
passed = 0
vote_count = 0
passed_counts = {}

for bl in bill_data:
    if count == 0:
        print(count)
        print(bl)
        #print()
    if count % 1000 == 0:
        print(count)  

    count+=1

    bill_id = str(bl['bill_id'])

    for s in bl['texts']:
        state_link = str(s['state_link'])
        doc_id = str(s['doc_id'])

    start = bl["session"]["year_start"]
    if start not in year_count:
        year_count[start] = 1
    else:
        year_count[start] += 1
    try: 
        does_it_pass = int(bl['status'] == '4')
        passed += does_it_pass#int(bl['votes'][0]['passed'])

        if start not in passed_counts:
            passed_counts[start] = does_it_pass
        else:
            passed_counts[start] += does_it_pass

        vote_count += 1
    except:
        continue
        
        
        
for bl in bill_data:
    if count == 0:
        print(count)
        print(bl) 
        
    count+=1
    does_it_pass = int(bl['status'] == '4')
    passed += does_it_pass
    
        
        


print(count)
print(year_count)
print(passed/count)
print(passed/vote_count)

0
{'_id': ObjectId('5e0c0a311e37b31b20fa7207'), 'bill_id': 668415, 'change_hash': 'c79a10f37f9af62998338d80c634ca3c', 'session_id': 1156, 'session': {'session_id': 1156, 'session_name': '114th Congress', 'session_title': '114th Congress', 'year_start': 2015, 'year_end': 2016, 'special': 0}, 'url': 'https://legiscan.com/US/bill/HB1/2015', 'state_link': 'https://www.congress.gov/bill/114th-congress/house-bill/1/all-info', 'completed': 0, 'status': 1, 'status_date': '2015-01-06', 'progress': [{'date': '2015-01-06', 'event': 1}], 'state': 'US', 'state_id': 52, 'bill_number': 'HB1', 'bill_type': 'B', 'bill_type_id': '1', 'body': 'H', 'body_id': 114, 'current_body': 'H', 'current_body_id': 114, 'title': 'Reserved for the Speaker.', 'description': 'Reserved for the Speaker.', 'committee': [], 'pending_committee_id': 0, 'history': [{'date': '2015-01-06', 'action': 'Introduced in House', 'chamber': 'H', 'chamber_id': 114, 'importance': 0}], 'sponsors': [{'people_id': 9383, 'person_hash': '0bgyx

In [6]:
# passed_789 = passed_counts[2017]+passed_counts[2018]+passed_counts[2019]
# total_789 = year_count[2017]+year_count[2018]+year_count[2019]

# passed_34 = passed_counts[2013]+passed_counts[2014]
# total_34 = year_count[2013]+year_count[2014]

# print("proportion passed 2013+2014:",passed_34/total_34)
# print('passed in 2013+2014:',passed_34)
# print('total in 2013+2014:',total_34)



In [7]:
import sys
sys.version_info

sys.version_info(major=3, minor=7, micro=6, releaselevel='final', serial=0)

In [8]:
#keywords we're looking for in bill events that allow us to simplify the event descriptions
#note: 'introduced and adopted' was not in the list but frequent in passing bills so i added it to the end
nh_status_keywords = ['amendment adopted','bill killed','children and family law','commerce'
,'committee','committee amendment','report','consumer affairs','criminal justice'
,'died on table','education','election law','energy','environment and agriculture'
,'executive departments and administration','executive session','finance','fish and game'
,'hearing','inexpedient to legislate','interim study'
,'internal affairs','introduced','judiciary','labor, industrial and rehabilitative services'
,'lay on table','legislative administration','majority committee report'
,'minority committee report','motion adopted','motion failed'
,'moved inexpedient to legislate','municipal and county government','natural resources'
,'ought to pass','ought to pass with amendment','public'
,'public works and highways','referred to','removed from consent'
,'resources, recreation and development','retained in committee','science, technology'
,'state-federal relations','to be introduced','transportation','vacated to','voice'
,'vote','ways and means','without objection','elderly affairs','workforce development'
,'municipal affairs','health','human services','veterans affairs','marine resources'
,'public safety','vacated and referred to','vacated from','enrolled','signed', 'special order'
,'subcommittee', 'work session','retained bill','cancelled','rescheduled','next session day'
,'beginning of regular calendar','law without signature', 'continued','recessed','vetoed'
,'veto sustained','floor amendment','house non-concurs with senate','senate non-concurs with house'
,'speaker appoints','speaker appoints alternates','request for committee of conference','accedes to'
,'requests cofc','request cofc','request committee of conference','requests committee of conference', 'introduced and adopted']

# passing keywords list
nh_passing_keywords =['ought to pass' ,'ought to pass with amendment'
                     ,'signed', 'law without signature', 'motion adopted', 'introducted and adopted']

In [16]:
def remove(value, deletechars):
    for c in deletechars:
        value = value.replace(c,'')
    return value

#reduces the event description on a bill into the key words and phrases
#defined in the classifier words list
def simplify_bill_event(bill_event, classifier_keywords):
    temp_stat = []
    for k in nh_status_keywords:
        pos = s.find(k)
        if pos != -1:
            temp_stat.append([k,pos])
    temp_stat.sort(key=lambda x: x[1])
    simplified_event = ' '.join(item[0] for item in temp_stat)
    return simplified_event

def replace(string, replace_this, with_this):
    new_string = string.split()
    for i, word in enumerate(string.split()):
        if word == replace_this:
            new_string[i] = with_this
    return " ".join(new_string)
        

def get_committee(first_action):
    first_action = first_action.split()
    committee = "None"
    for i, word in enumerate(first_action):
        if first_action[i]=="to" and first_action[i-1]=="Referred":
            committee = first_action[i+2:i+9]
            committee = " ".join(committee)
            break
    for i,letter in enumerate(committee):
        if letter == ";":
            committee = committee[0:i]
        elif letter == ",":
            committee = committee[0:i]
        elif letter == ".":
            committee = committee[0:i]    
    committee = replace(committee, '&', 'and')
    return committee
    
            
print(get_committee('Introduced and Referred to Ways & Means, Senate Journal 6'))
#print(replace('Introduced & Referred to Ways & Means, Senate Journal 6', '&', 'and'))

#'Introduced and Referred to Ways and Means; House Journal  12, PG. 212 '
#'Introduced and Referred to Commerce, Senate Journal 3, Pg.35'
#'Introduced and Referred to Ways and Means, Senate Journal 6'
#

and Means


In [22]:
required_length = len(bill_data)

#define the lists that will make up the dictionary loaded into pandas---------------
bill_id_list = [] #keep
time_t_list = [] #keep
hist_list = [[]] #keep
next_hist_list = [[]] #keep
event_list = [] #keep
next_event_list=[] #keep, is target
sponsor_list = [] #keep
sponsor_vec_list = []
sponsor_vec2=[]
bill_category_list =[]
event_out_desc_list = []
event_out_metric_list = []
doc_id_list = []
year_list = [] #keep
session_list = [] #keep
chamber_list = [] #keep
win_state_list=[] #keep
overall_passed_list=[] #keep, is target
public_sentiment=[] #unused, maybe eventually
public_knowledge=[] #unused, maybe eventually


#add the features from bill_features.ipynb too
title_list = []
proposed_chamber_list = []
primary_name_list = []
primary_id_list = []
primary_party_list = []
number_sponsors_list = []
committee_introduced_list = []
month_introduced_list = []

#newest features
bill_type_list = [] #e.g. "CA", constitutional amendment
numeric_date_list = [] #year+((month-1)/12)+((day-1)/31)


#Grab the lists
for b in bill_data:
    #assign temporary value for everything
    #temp_hist_list=[]
    #temp_next_hist_list = []
    bill_temp = b['bill_id']
    #year_temp = b['session']['year_end']
    session_id_temp = b['session']['session_id']
    
    title_temp = b['title']
    proposed_chamber_temp = str(b['body'])
    try:
        win_state_temp = int(b['status'] == '4')
    except:
        win_state_temp = 0
        
    try:
        primary_id_temp = b['sponsors'][0]['people_id']
        primary_name_temp = b['sponsors'][0]['name']
        primary_party_temp = b['sponsors'][0]['party']
    except:
        primary_id_temp = 'None'
        primary_name_temp = 'None'
        primary_party_temp = 'None'   
        
    number_sponsors_temp = len(b['sponsors'])
    try:
        committee_introduced_temp = get_committee(b['history'][1]['action'])
    except:
        committee_introduced_temp = 'None'
    
    bill_time_stamp = str(b['status_date'])
    if bill_time_stamp == "None":
        year_temp = -1
        month_introduced_temp = -1
        numeric_day_temp = -1
    else:
        _time = datetime.strptime(bill_time_stamp, "%Y-%m-%d")
        year_temp = _time.strftime("%Y")
        month_introduced_temp = _time.strftime("%m")
        day = _time.strftime("%d")
        numeric_day_temp = float(year_temp) + (float(month_introduced_temp)-1)/12 + (float(day)-1)/(31*12)
    
    bill_type_temp = b['bill_type']
    

    #as time, t, is based on the events that occur on a bill, the lists will include duplicates, as necessary, that align to the number of t instances
    #for i,h in enumerate(b['history'],start=1):
    #temp_hist_list.append(remove(str(h['action']).lower(),'\/:*?"<>|0123456789.'))
    #chamber_list.append(str(h['chamber']))
    #time_t_list.append(i) 
    bill_id_list.append(bill_temp)
    year_list.append(year_temp)
    session_list.append(session_id_temp)

    #new features here
    title_list.append(title_temp)
    proposed_chamber_list.append(proposed_chamber_temp)
    primary_name_list.append(primary_name_temp)
    primary_id_list.append(primary_id_temp)
    primary_party_list.append(primary_party_temp)
    number_sponsors_list.append(number_sponsors_temp)
    committee_introduced_list.append(committee_introduced_temp)
    month_introduced_list.append(month_introduced_temp)
    #newest features
    bill_type_list.append(bill_type_temp)
    numeric_date_list.append(numeric_day_temp)
    win_state_list.append(win_state_temp)

    sponsor_vector = 0.0000
    total_sponsors = 0.0
    #grab the next event, which is our label for the MN_NB
#     try:
#         next_event = b['history'][i]
#     except:
#         next_event = h

    #temp_next_hist_list.append(remove(str(next_event['action']).lower(),'\/:*?"<>|0123456789.'))

    for x in b['sponsors']: #this should probably go outside above the previous for loop, but I don't want to break anything
        if x['party'] == 'D':
            sponsor_vector+=0
        elif x['party'] == 'R':
            sponsor_vector+=1
        total_sponsors += 1
    if total_sponsors == 0:
        sponsor_list.append(0.5)
    else:
        sponsor_list.append(sponsor_vector/total_sponsors)   
#---------------------------------------------------------------------------------------------------------    
    #grab the associated doc_ids to pull in the cluster information
    #use the date of the bill revisions to determine
#---------------------------------------------------------------------------------------------------------
#     if len(b['texts']) == 0:
#         doc_id_list.append(0)

#     else:
#         for j,d in enumerate(b['texts']):

#             doc_date_str = d['date']

#             #determine the next date in the list to create a date range to
#             #check for
#             try:
#                 next_doc = b['texts'][j+1]
#                 doc_next_date = dateutil.parser.isoparse((next_doc['date'])).date()

#             except:
#                 doc_next_date = date(2100,1,1)

#             #hist_date_string = h['date']
#             hist_date = dateutil.parser.isoparse(hist_date_string).date()

#             if doc_date_str == '0000-00-00':
#                 doc_date = date(2000,1,1)
#             else:
#                 doc_date = dateutil.parser.isoparse(doc_date_str).date()

#             if hist_date >= doc_date and hist_date < doc_next_date:
#                 doc_id_list.append(d['doc_id'])
    #---------------------------------------------------------------------------------------------------------
    #hist_list.append(temp_hist_list)
    #next_hist_list.append(temp_next_hist_list)
    

In [13]:
#process the hist_list to create the simplified event list (use keywords to simplify event descriptions)
#uses the UDF
# event_list = []
# next_event_list=[]

# for h in hist_list:
#     for s in h:
#         event_list.append(simplify_bill_event(s,nh_status_keywords))
# for h in next_hist_list:
#     for s in h:
#         next_event_list.append(simplify_bill_event(s,nh_status_keywords))
# print(len(event_list), len(next_event_list))

124345 124345


In [24]:
length_check = [chamber_list, time_t_list,
                bill_id_list, year_list, session_list, title_list,
                proposed_chamber_list, primary_name_list, primary_id_list, primary_party_list,
                number_sponsors_list, committee_introduced_list, month_introduced_list, bill_type_list,
                numeric_date_list, win_state_list]          
for i in (length_check):
    print(len(i))

0
0
69676
69676
69676
69676
69676
69676
69676
69676
69676
69676
69676
69676
69676
69676


In [19]:
print(committee_introduced_list[1000:1010])

['None', "House Committee on Veterans' Affairs", 'House Committee on Natural Resources', 'House Committee on Ways and Means', 'House Committee on Natural Resources', 'House Committee on Ways and Means', "House Committee on Veterans' Affairs", 'House Committee on Armed Services', 'House Committee on Ways and Means', 'Committee on Ways and Means']


In [20]:
leg_dict = {'bill_id':bill_id_list
            ,'numeric_date':numeric_date_list
            #,'time_t':time_t_list
            #,'event':event_list
            ,'sponsor_vec':sponsor_list
            ,'session':session_list
            ,'year':year_list
            #,'chamber':chamber_list
            ,'bill_type':bill_type_list
            ,'title':title_list
            ,'proposed_chamber':proposed_chamber_list
            ,'primary_name':primary_name_list
            ,'primary_id':primary_id_list
            ,'primary_party':primary_party_list
            ,'number_sponsors':number_sponsors_list
            ,'committee_introduced':committee_introduced_list
            ,'month_introduced':month_introduced_list}

In [25]:
#Define the win state
print(sum(win_state_list)/len(win_state_list))

# x=0
# for b in leg_dict['event']:
#     if b in nh_passing_keywords:
#         x+=1
#         win_state_list.append(1)
#     else:   
#         win_state_list.append(0)
# print(x)

# #Define a variable which has 1 at any time stage to use for overall model
# #might want to change this?
# temp_var=[0]*len(bill_id_list)
# current_bill = 1
# cur_id = bill_id_list[0]

# for y in range(0, len(bill_id_list)): 
#     if cur_id == bill_id_list[y]:
#         temp_var[y] = current_bill
#     else:
#         current_bill = current_bill + 1   
#         temp_var[y] = current_bill      
#     cur_id = bill_id_list[y] 
# print(temp_var[len(temp_var)-2])


# overall_passed_list = [0]*len(bill_id_list)

# for x in range(len(temp_var)-2,-1,-1):   
#     if win_state_list[x] == 1:
#         overall_passed_list[x] = 1
        
#     elif overall_passed_list[x+1] == 1 and temp_var[x] == temp_var[x+1]:
#         overall_passed_list[x] = 1
        
#     else:
#         overall_passed_list[x] = 0
        

0.03278029737642804


In [24]:
#event_target = next_event_list #what we will try to predict when predicting the next state
#overall_passed_list is what we will try to predict when predicting overall probability

In [26]:
#output everything
with open('FedWinList09_20.list','wb') as f:
    pickle.dump(win_state_list, f)
    
#with open('FedStateTargetList09_20.list','wb') as f:
#    pickle.dump(event_target, f)

overall_leg_DF = pd.DataFrame(leg_dict)    
overall_leg_DF.to_csv('FedLegOverallData09_20.csv')